# Most Dangerous Streets - Crash Leaderboard

Compare the most recent month to the all-time numbers to see which streets got safer. 

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2020,7,1))
end_date = cda.most_recent_crash_timestamp()

all_time = cda.street_metrics(start_date, end_date)

In [4]:
start_date = pytz.timezone('America/Denver').localize(datetime(2023,6,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,7,1))

last_month = cda.street_metrics(start_date, end_date)

In [5]:
# A street must have at least 100 crashes all-time to be in the comparison
all_time = all_time[all_time.num_crashes >= 100].copy()

In [6]:
all_time

,gid,lrsroute,fullname,length_miles,street_line,num_crashes,num_sbi,num_fatality,num_sbi_or_fatality,days_in_data,days_between_crashes,crashes_per_mile_per_week
1585,3900,SPEERN BD,N SPEER BLVD,2.634723,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",705,22,5,27,1098,1.557447,1.705885
1764,4229,22ND ST,22ND ST,1.079568,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",234,4,2,6,1098,4.692308,1.381852
2411,5587,PARK AV,PARK AVE W,1.809664,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",377,12,2,14,1098,2.912467,1.328125
1015,1957,MARKET ST,MARKET ST,0.920197,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",185,7,0,7,1098,5.935135,1.281700
756,1452,ESPEERN BD,E SPEER BLVD,1.060124,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",209,6,2,8,1098,5.253589,1.256856
...,...,...,...,...,...,...,...,...,...,...,...,...
3182,7738,SLOGAN ST,S LOGAN ST,3.377398,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",128,2,0,2,1098,8.578125,0.241615
332,572,56TH AV,E 56TH AVE,9.027374,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",318,12,3,15,1098,3.452830,0.224575
113,174,23RD AV1,E 23RD AVE,4.660338,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",155,4,1,5,1098,7.083871,0.212036
175,291,35TH AV1,E 35TH AVE,4.421519,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",139,3,1,4,1098,7.899281,0.200419


In [7]:
comparison = pd.merge(
    all_time
    , last_month
    , how='inner'
    , on=['gid', 'lrsroute', 'fullname', 'length_miles','street_line']
    , suffixes=['_all_time', '_last_month']
)

In [8]:
comparison['diff_pmpw'] = (
    comparison['crashes_per_mile_per_week_last_month']
    - comparison['crashes_per_mile_per_week_all_time']
)

In [9]:
comparison['expected_crashes'] = (
    comparison['length_miles']
    * comparison['crashes_per_mile_per_week_all_time']
    * (comparison['days_in_data_last_month'] / 7)
)

comparison['diff_expected_crashes'] = comparison['num_crashes_last_month'] - comparison['expected_crashes']

## Difference in Crash Rate per Mile

In [10]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_pmpw').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_pmpw').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
comparison.sort_values(by='diff_pmpw')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'num_crashes_all_time'
    , 'num_crashes_last_month'
    , 'expected_crashes'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,num_crashes_all_time,num_crashes_last_month,expected_crashes,diff_expected_crashes
20,1122,N CHAMBERS RD,1.758017,0.899341,0.132725,-0.766615,248,1,6.775956,-5.775956
41,1415,N GRANT ST,2.162158,0.633938,0.107917,-0.526021,215,1,5.874317,-4.874317
42,156,E 1ST AVE,1.225738,0.629337,0.190361,-0.438976,121,1,3.306011,-2.306011
10,5588,PARK AVE W,0.651755,1.144450,0.716015,-0.428435,117,2,3.196721,-1.196721
9,1714,N LINCOLN ST,2.165535,1.189356,0.861989,-0.327367,404,8,11.038251,-3.038251
...,...,...,...,...,...,...,...,...,...,...
17,2127,N PEORIA ST,2.089566,0.985467,1.674989,0.689522,323,15,8.825137,6.174863
21,2941,N TOWER RD,1.027545,0.887220,1.589550,0.702330,143,7,3.907104,3.092896
3,1957,MARKET ST,0.920197,1.281700,2.028551,0.746850,185,8,5.054645,2.945355
33,1799,N JOSEPHINE ST,1.876373,0.716901,1.492241,0.775340,211,12,5.765027,6.234973


## Difference in Number of Expected Crashes

In [12]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_expected_crashes').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_expected_crashes').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [13]:
comparison.sort_values(by='diff_expected_crashes')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'expected_crashes'
    , 'num_crashes_last_month'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,expected_crashes,num_crashes_last_month,diff_expected_crashes
20,1122,N CHAMBERS RD,1.758017,0.899341,0.132725,-0.766615,6.775956,1,-5.775956
41,1415,N GRANT ST,2.162158,0.633938,0.107917,-0.526021,5.874317,1,-4.874317
26,4689,N FEDERAL BLVD,5.367276,0.852837,0.652100,-0.200737,19.617486,15,-4.617486
27,4592,S COLORADO BLVD,4.362694,0.827099,0.588321,-0.238777,15.464481,11,-4.464481
18,5379,W COLFAX AVE,3.567320,0.945386,0.719494,-0.225892,14.453552,11,-3.453552
...,...,...,...,...,...,...,...,...,...
33,1799,N JOSEPHINE ST,1.876373,0.716901,1.492241,0.775340,5.765027,12,6.234973
92,920,N CENTRAL PARK BLVD,4.490628,0.401768,0.727441,0.325673,7.732240,14,6.267760
12,4216,17TH ST,0.928871,1.043240,2.763213,1.719973,4.153005,11,6.846995
49,125,E 14TH AVE,5.479802,0.588683,0.894193,0.305510,13.825137,21,7.174863


In [14]:
comparison = comparison.sort_values(by='diff_expected_crashes')

comparison.loc[comparison['diff_expected_crashes'].rank() <= 5, 'Leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank() <= 5].apply(
        lambda row: f'{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5, 'Leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5].apply(
        lambda row: f'+{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

# comparison['Expected Crashes'] = comparison.expected_c

In [21]:
comparison['street_name'] = comparison['fullname'].str.title()
comparison.loc[
    comparison.leaderboard.notnull()
    , ['street_name', 'expected_crashes', 'num_crashes_last_month', 'leaderboard']
].style.format('{:.1f}', subset='expected_crashes')

,street_name,expected_crashes,num_crashes_last_month,leaderboard
20,N Chambers Rd,6.8,1,-6 crashes
41,N Grant St,5.9,1,-5 crashes
26,N Federal Blvd,19.6,15,-5 crashes
27,S Colorado Blvd,15.5,11,-4 crashes
18,W Colfax Ave,14.5,11,-3 crashes
33,N Josephine St,5.8,12,+6 crashes
92,N Central Park Blvd,7.7,14,+6 crashes
12,17Th St,4.2,11,+7 crashes
49,E 14Th Ave,13.8,21,+7 crashes
0,N Speer Blvd,19.3,27,+8 crashes
